In [1]:
import tensorflow as tf
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Dense,Add,add,BatchNormalization,Conv1D,concatenate,Flatten 
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications import MobileNet
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python import keras
from IPython.display import SVG
from tensorflow.python.keras.utils import plot_model
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import random
import os
import glob

C:\Users\Siby\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
folder_name = './data/power_data'
folder_name = './data/earthquake_data'
file_names = glob.glob(os.path.join(folder_name,'*.csv'))  #List only csv files
file_names[0]

'./data/earthquake_data\\earthquake.csv'

In [4]:
def _get_input_data_for_dataset(file_name): #rows = 100

    #df_input=pd.read_csv(file_name.decode(),usecols=['Wind_MWh'])#, nrows=rows
    df_input=pd.read_csv(file_name.decode(),usecols=['Latitude','Longitude','Depth'])#, nrows=rows
        
   
    X_data = df_input.as_matrix()
   
    return X_data.astype('float32', copy=False)

def _get_target_data_for_dataset(file_name): #rows = 100

    #df_input=pd.read_csv(file_name.decode(),usecols=['Actual_Load_MWh'])#, nrows=rows
    df_input=pd.read_csv(file_name.decode(),usecols=['Magnitude'])#, nrows=rows
        
   
    X_data = df_input.as_matrix()
   
    return X_data.astype('float32', copy=False)

def _get_validation_data_for_dataset(file_name): 
    valid_batch_size = 1000
    #df_input=pd.read_csv(file_name.decode(),usecols=['Actual_Load_MWh'])#, nrows=rows
    df_input=pd.read_csv(file_name,usecols=['Latitude','Longitude','Depth'],skiprows=range(1, 20000) )#
    df_target=pd.read_csv(file_name,usecols=['Magnitude'],skiprows=range(1, 20000) )    
   
    X_data = df_input.as_matrix()
    Y_data = df_target.as_matrix()
   
    return (X_data.astype('float32', copy=False),Y_data.astype('float32', copy=False))

In [5]:
train_batch_size = 256
X_dataset = tf.data.Dataset.from_tensor_slices(file_names)
X_dataset = X_dataset.flat_map(lambda file_name: tf.data.Dataset.from_tensor_slices(
                                tf.reshape(tf.py_func(_get_input_data_for_dataset,[file_name], tf.float32),[-1,3])))

X_dataset = X_dataset.batch(train_batch_size).repeat()
X_iter = X_dataset.make_one_shot_iterator()
X_batch = X_iter.get_next()

Y_dataset = tf.data.Dataset.from_tensor_slices(file_names)
Y_dataset = Y_dataset.flat_map(lambda file_name: tf.data.Dataset.from_tensor_slices(
                                tf.reshape(tf.py_func(_get_target_data_for_dataset,[file_name], tf.float32),[-1,1])))
Y_dataset = Y_dataset.batch(train_batch_size).repeat()
Y_iter = Y_dataset.make_one_shot_iterator()
Y_batch = Y_iter.get_next()

XY_dataset = tf.data.Dataset.from_tensor_slices(_get_validation_data_for_dataset(file_names[0]))
#XY_dataset = XY_dataset.flat_map(lambda file_name: tf.data.Dataset.from_tensor_slices(
#                                tuple(tf.py_func(_get_validation_data_for_dataset,[file_name], [tf.float32,tf.float32]))))  
XY_dataset = XY_dataset.batch(train_batch_size).repeat()
XY_iter = XY_dataset.make_one_shot_iterator()
XY_batch = XY_iter.get_next()

In [6]:
class NN_model():
    """ Class for neural network model."""
    
    __counter = 0
    max_layers = 5
    max_neurons = 15
    
    def __init__(self,layers = 1, neurons = 3): 
        type(self).__counter += 1
        self.layers = layers
        self.neurons = neurons
        self.model_ID = 'model_'+str(self.__counter)
        self.make_FFNN(X_batch,Y_batch)
    def __del__(self):
        type(self).__counter -= 1
    
    @staticmethod
    def NN_instances():
        return NN_model.__counter
    
    def make_FFNN(self,X_batch,Y_batch):
        X_features = X_batch.get_shape()[1].value
        Y_features = Y_batch.get_shape()[1].value
        input_X1 = Input(tensor= X_batch,name='input_X1')
        norm_X1 = tf.keras.layers.BatchNormalization(name='Batch_norm')(input_X1)
        
        for i in range(self.layers):
            if i == 0:
                y1 = Dense(units= self.neurons, activation='relu',name='layer_FC'+str(i+1))(norm_X1)
            elif i>=1:
                y1 = Dense(units= self.neurons, activation='relu',name='layer_FC'+str(i+1))(y1)
        
        y1 = Dense(units=Y_features,name='output_layer')(y1)
        self.model = Model(inputs=input_X1,outputs=y1,name=self.model_ID)
        self.model.compile('adam', 'mse', target_tensors=[Y_batch])
        self.fitness = self.get_losses()
        
        return self.model
    
    def plot_model(self):
       
        plot_model(self.model, show_shapes = True,to_file=self.model_ID+'.svg')
        SVG('./'+self.model_ID+'.svg')
    
    def show_model(self):
        """Show NN model details."""

        print("Model ID:{}, Fitness:{:3f}, Layers:{}, Neurons:{}".format(self.model_ID,self.fitness,self.layers,self.neurons))
    
    def get_losses(self):
        """Train NN model details and get loss."""
        print("Training a new NN model with ID:{} having {} layers and {} neurons per layer.".format(self.model.name,self.layers,self.neurons))
        #return self.model.evaluate(steps=10)
        return self.model.fit(steps_per_epoch=50, epochs=5,verbose=0).history['loss'][-1] #, validation_split=0.2,validation_data =XY

In [7]:
class PSO_NN_Particle(NN_model):
    """Class to encapsulate particle in swarm."""
    
    __counter = 0
    c1 = 0.1  #PSO inertial hyperparametre
    
    def __init__(self): 
        type(self).__counter += 1
        layers =  random.randint(1, NN_model.max_layers)
        neurons =  random.randint(1, NN_model.max_neurons) 
        super().__init__(layers,neurons) 
        
        #self.make_FFNN(X_batch,Y_batch)
        self.particle_ID = 'particle_'+str(self.__counter)
        
        self.best_fitness = self.fitness
        self.best_layers = layers
        self.best_neurons = neurons
        
        self.velocity_layers = 0
        self.velocity_neurons = 0
    
    def __del__(self):
        type(self).__counter -= 1
    
    def show_particle(self):
        """Show contents of swarm."""
        
        print("Particle ID:{}, Best fitness:{:3f}, Best layers:{}, Best neurons:{}".\
            format(self.particle_ID,self.best_fitness,self.best_layers,self.best_neurons))
        super().show_model() 
    
    def velocity_change_inertial(self):
        """Inertial component of particle velocity."""
        
        self.velocity_change_inertial_layers =  (self.best_layers - self.layers)*self.c1*random.random()
        self.velocity_change_inertial_neurons = (self.best_neurons - self.neurons)*self.c1*random.random()
     
    def update_particle_velocity(self):
        """Function to update particle velocity."""       
           
        self.velocity_layers = self.velocity_layers +   self.velocity_change_inertial_layers +\
                                                        self.velocity_change_global_layers
        self.velocity_neurons = self.velocity_neurons + self.velocity_change_inertial_neurons +\
                                                        self.velocity_change_global_neurons
    
    def update_particle_best(self):
        """Update particle best fitness."""
       
        if self.fitness < self.best_fitness:
            self.best_fitness = self.fitness
            self.best_layers = self.layers
            self.best_neurons = self.neurons
    
    def update_particle(self):
        """Update particle with new NN model and fitness."""
        
        #self.layers =  min(self.max_layers,int(self.layers +   self.velocity_layers))
        #self.neurons = min(self.max_neurons,int(self.neurons +   self.velocity_neurons))
        #self.make_FFNN(X_batch,Y_batch)
        layers =  min(self.max_layers,int(self.layers +   self.velocity_layers))
        neurons = min(self.max_neurons,int(self.neurons +   self.velocity_neurons))
        
        if layers != self.layers or neurons != self.neurons:
            print("Updating model for particle {}.".format(self.particle_ID))    
            super().__init__(layers,neurons)       
            self.update_particle_best()

In [8]:
class PSO_NN_Swarm(PSO_NN_Particle):
    """Class to encapsulate PSO algorithm."""
    
    __counter = 0
   
    c2 =0.1  #Global hyper parameter
    
    def __init__(self,swarm_size=2,N= 2): 
        type(self).__counter += 1
        self.swarm_size = swarm_size
        self.N = N
        self.swarm_ID = 'swarm_'+str(self.__counter)
        self.swarm = []
        
    def __del__(self):
        type(self).__counter -= 1
    
    @staticmethod
    def swarm_instances():
        return PSO_NN_Swarm.__counter
    
    def populate_swarm(self):
        """Populate swarm with NN models."""

        for particle in range(self.swarm_size):
            self.swarm.append(PSO_NN_Particle())
        self.best_fitness = self.swarm[0].best_fitness  #Initialize with values of first particle
        self.best_layers = self.swarm[0].layers
        self.best_neurons = self.swarm[0].neurons
        self.best_particle_ID = self.swarm[0].particle_ID
        self.update_global_best()
    
    def show_swarm(self):
        """Show contents of swarm."""
        
        print("Swarm ID:{}, Best fitness:{}, Best layers:{}, Best neurons:{}, Best particle:{}"
              .format(self.swarm_ID,self.best_fitness,self.best_layers,self.best_neurons,self.best_particle_ID))
        for particle in self.swarm:
            particle.show_particle()
        
    
    def update_global_best(self):
        """Find particle with best fitness in swarm."""
        
        for particle in self.swarm:
            if particle.best_fitness < self.best_fitness:
                self.best_fitness = particle.best_fitness
                self.best_layers = particle.best_layers
                self.best_neurons = particle.best_neurons
                self.best_particle_ID = particle.particle_ID
    
    def velocity_change_global(self,particle):
        """Global component of particle velocity."""
        
        particle.velocity_change_global_layers =  (self.best_layers - particle.layers)*self.c2*random.random()
        particle.velocity_change_global_neurons = (self.best_neurons - particle.neurons)*self.c2*random.random()
    
   
    def PSO_algorithm(self):
        """Apply PSO algorithm on swarm."""
        
        for i in range(self.N):
            print("Iteration:{}".format(i))
            #self.update_global_best()
            
            for particle in self.swarm:
                particle.velocity_change_inertial()
                self.velocity_change_global(particle)
                particle.update_particle_velocity()  
                particle.update_particle()
            self.update_global_best()
            self.show_swarm()

In [9]:
swarm1 = PSO_NN_Swarm(swarm_size=5,N=15)

In [10]:
swarm1.populate_swarm()

Training a new NN model with ID:model_1 having 4 layers and 7 neurons per layer.
Training a new NN model with ID:model_2 having 2 layers and 14 neurons per layer.
Training a new NN model with ID:model_3 having 5 layers and 14 neurons per layer.
Training a new NN model with ID:model_4 having 5 layers and 5 neurons per layer.
Training a new NN model with ID:model_5 having 2 layers and 13 neurons per layer.


In [ ]:
swarm1.swarm

In [11]:
swarm1.show_swarm()

Swarm ID:swarm_1, Best fitness:0.991333931684494, Best layers:4, Best neurons:7, Best particle:particle_1
Particle ID:particle_1, Best fitness:0.991334, Best layers:4, Best neurons:7
Model ID:model_1, Fitness:0.991334, Layers:4, Neurons:7
Particle ID:particle_2, Best fitness:1.587368, Best layers:2, Best neurons:14
Model ID:model_2, Fitness:1.587368, Layers:2, Neurons:14
Particle ID:particle_3, Best fitness:1.130680, Best layers:5, Best neurons:14
Model ID:model_3, Fitness:1.130680, Layers:5, Neurons:14
Particle ID:particle_4, Best fitness:2.204261, Best layers:5, Best neurons:5
Model ID:model_4, Fitness:2.204261, Layers:5, Neurons:5
Particle ID:particle_5, Best fitness:1.433454, Best layers:2, Best neurons:13
Model ID:model_5, Fitness:1.433454, Layers:2, Neurons:13


In [12]:
swarm1.PSO_algorithm()

Iteration:0
Updating model for particle particle_2.
Training a new NN model with ID:model_6 having 2 layers and 13 neurons per layer.
Updating model for particle particle_3.
Training a new NN model with ID:model_7 having 4 layers and 13 neurons per layer.
Updating model for particle particle_4.
Training a new NN model with ID:model_8 having 4 layers and 5 neurons per layer.
Updating model for particle particle_5.
Training a new NN model with ID:model_9 having 2 layers and 12 neurons per layer.
Swarm ID:swarm_1, Best fitness:0.991333931684494, Best layers:4, Best neurons:7, Best particle:particle_1
Particle ID:particle_1, Best fitness:0.991334, Best layers:4, Best neurons:7
Model ID:model_1, Fitness:0.991334, Layers:4, Neurons:7
Particle ID:particle_2, Best fitness:1.587368, Best layers:2, Best neurons:14
Model ID:model_6, Fitness:1.631794, Layers:2, Neurons:13
Particle ID:particle_3, Best fitness:1.017958, Best layers:4, Best neurons:13
Model ID:model_7, Fitness:1.017958, Layers:4, Neu

Updating model for particle particle_3.
Training a new NN model with ID:model_28 having 3 layers and 6 neurons per layer.
Swarm ID:swarm_1, Best fitness:0.991333931684494, Best layers:4, Best neurons:7, Best particle:particle_1
Particle ID:particle_1, Best fitness:0.991334, Best layers:4, Best neurons:7
Model ID:model_1, Fitness:0.991334, Layers:4, Neurons:7
Particle ID:particle_2, Best fitness:1.587368, Best layers:2, Best neurons:14
Model ID:model_27, Fitness:2.128760, Layers:3, Neurons:7
Particle ID:particle_3, Best fitness:1.008346, Best layers:3, Best neurons:12
Model ID:model_28, Fitness:5.998410, Layers:3, Neurons:6
Particle ID:particle_4, Best fitness:2.204261, Best layers:5, Best neurons:5
Model ID:model_8, Fitness:32.055486, Layers:4, Neurons:5
Particle ID:particle_5, Best fitness:1.433454, Best layers:2, Best neurons:13
Model ID:model_24, Fitness:19.077455, Layers:2, Neurons:7
Iteration:8
Updating model for particle particle_2.
Training a new NN model with ID:model_29 having